In [ ]:
import pandas as pd
import numpy as np
# 适合在ipython notebook里面使用
# %matplotlib inline

# 可以设置不同的绘图风格
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# import seaborn as sns
# sns.set()

# https://matplotlib.org/3.1.0/api/matplotlib_configuration_api.html#matplotlib.RcParams
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
plt.rcParams['figure.figsize'] = (20, 10)
# x,y轴字体大小，x属性字体大小，每个曲线标题字体大小
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['legend.fontsize'] = 18

# 或者使用下面这个方式
# plt.rc('font', **{'sans-serif': ['SimHei']})
# plt.rc('axes', **{'unicode_minus': False})
# plt.rc('figure', **{'figsize': (20, 10)})

# np/df 数值显示方式
np.set_printoptions(precision=4)
pd.set_option('display.width', 5000)
pd.set_option('display.max_columns', 60)

In [ ]:
import json
data = json.load(open('output.json'))['benchmarks']
def normalize(d):
    name = d['name']
    name, size = name.split('/')[:2]
    size = int(size)
    cpu_time = float(d['cpu_time'])
    t = d['threads']
    return {'name': name, 'size': size, 'cpu': cpu_time, 'threads': t}
data = [normalize(x) for x in data]
sizes = list(set((x['size'] for x in data)))
sizes.sort()
names = list(set((x['name'] for x in data)))
names.sort()

In [ ]:
def doplot(size, ss):    
    def fun(ss, t):
        a = [x for x in ss if x['threads'] == t]
        a.sort(key = lambda x: x['cpu'])   
        names = [x['name'] for x in a]
        times = [x['cpu'] for x in a]
        plt.barh(names, times)        
    
    plt.figure(figsize = (40,20))
    
    t = 1
    plt.subplot(1, 2, 1)    
    fun(ss, t)
    plt.ylabel('cpu ns')
    plt.title('Memcpy on size %d - Threads = %d' % (size, t))
    
    t = 8
    plt.subplot(1,2,2)
    fun(ss, t)
    plt.ylabel('cpu ns')
    plt.title('Memcpy on size %d - Threads = %d' % (size, t))
    
    
def doplotdata(data, size):
    ss = [x for x in data if x['size'] == size]    
    doplot(size, ss)
    plt.savefig('memcpy_size_%d.png' % size)
    

In [ ]:
# output a lot of images files
if False:
    for s in sizes:
    doplotdata(data, s)

In [ ]:
def size2text(size):
    if size < 1024: return str(size)
    if size < 1024 * 1024: return str(size / 1024) + 'KB'
    return str(size / 1024 / 1024) + 'MB'

markers = 'o*+xD'
dolog = False

def docomp(data, t, output):
    plt.figure(figsize = (100,20))
    for i, name in enumerate(names):
        if name == 'memcpy_gutil': continue
        ss = [x for x in data if x['name'] == name and x['threads'] == t]
        xs = [x['size'] for x in ss]
        ys = [x['cpu'] for x in ss]
        if dolog:
            ys = np.log2(ys)        
        plt.plot([size2text(x) for x in xs], ys, markers[i]+'-', label =name.replace('_', '-'),linewidth=5, markersize=20)
    if dolog:
        plt.ylabel('CPU ns(log2)')
    else:
        plt.ylabel('CPU ns')
    plt.legend()
    plt.title('Thread = %d' % (t))
    plt.savefig(output)

docomp(data,  1, 'memcpy-thread-1.png')
docomp(data, 8, 'memcpy-thread-8.png')